# The Validation Experiment Design

## 1.1 Building the minimal transformer

In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Dict, List

In [3]:
@dataclass

# keeping everything tiny, for now
class ModelConfig:
    d_model: int = 32
    n_layers: int = 2
    n_heads: int = 2
    d_ff: int = 64
    vocab_size: int = 20
    max_seq_len: int = 10